In [16]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

from dcopf_utils import build_gurobi_model, build_osqp_model
import time
from dcopf_utils import DcopfProblem
from utils import build_ipopt

In [32]:
import scipy.io as spio

filepath = "datasets/dcopf/qp20_10_1_1.mat"
data = spio.loadmat(filepath)
b = data['beq'].squeeze()
# ranfomly generate X as numsamples of random perturbation of b by factors randomly generated from [0.9, 1.1]
num_samples = 10000
X = np.random.rand(num_samples, b.shape[0]) * 0.1 + 0.95
b = b.reshape(1, -1)
X = X * b
print(X.shape)
data = {'Q':data['H'], 'p':data['f'].squeeze(), 'A':data['Aeq'], 'X':X, 'G':data['A'], 'h':data['b'].squeeze(), 'Lb':data['LB'].squeeze(), 'Ub':data['UB'].squeeze(), 'Y':[]}
data = DcopfProblem(data, valid_frac=0.0833, test_frac=0.001)

(10000, 4)


In [33]:
from model_utils import NNSolver_eq_proj




arg_path = "results/DcopfProblem-20-10-4-10000/method_eq_proj/5df6812307c6ef62caf1bb1d7e9492a938ae32d1/1714051827-5459692/args.dict"
args = pickle.load(open(arg_path, 'rb'))

model = NNSolver_eq_proj(data, args)
model.to('cpu')
model_path = "/home/jxxiong/A-xjx/DC3/results/DcopfProblem-20-10-4-10000/method_eq_proj/5df6812307c6ef62caf1bb1d7e9492a938ae32d1/1714051827-5459692/solver_net.dict"
model.load_state_dict(torch.load(model_path))

model.eval()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [34]:
DEVICE = 'cpu'
model.to(DEVICE)
for attr in dir(data):
    var = getattr(data, attr)
    if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
        try:
            setattr(data, attr, var.to(DEVICE))
        except AttributeError:
            pass

In [35]:
A = data.A
G = data.G
h = data.h
Q = data.Q
p = data.p
Lb = data.Lb
Ub = data.Ub

A_np = data.A_np
G_np = data.G_np
h_np = data.h_np
Q_np = data.Q_np
p_np = data.p_np
Lb_np = data.Lb.cpu().numpy()
Ub_np = data.Ub.cpu().numpy()

X = data.X
X_np = data.X_np

num_var = A.shape[1]
num_eq = A.shape[0]
num_ineq = G.shape[0]

In [36]:
p.max()

tensor(3.9517)

In [37]:
X.shape

torch.Size([10000, 4])

In [38]:
idx = 1

In [39]:
# full_model = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
# start_time = time.time()
# r = full_model.solve()
# print("OSQP time: ", time.time() - start_time)
# print(r.info.obj_val)
# x_full = r.x

In [40]:
# Lb_np_reduced = Lb_np.copy()
# Lb_np_reduced[inactive_lower_bounds] = -np.inf
# Ub_np_reduced = Ub_np.copy()
# Ub_np_reduced[inactive_upper_bounds] = np.inf

# reduced_model = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np_reduced, Ub_np_reduced)
# start_time = time.time()
# r = reduced_model.solve()
# print("OSQP reduced time: ", time.time() - start_time)
# print(r.info.obj_val)
# x_reduced = r.x

In [41]:
def get_init(A, x):
    y0 = np.linalg.pinv(A)@x
    return y0

full_model = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
start_time = time.time()
optimal_x_full, info_full = full_model.solve(get_init(A_np, X_np[idx]))
end_time = time.time()
time_full = end_time - start_time
print(info_full['obj_val'], time_full)

-1.3693567536045435e+42 0.037088632583618164


In [42]:
model_presolve = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
%time model_presolve = model_presolve.presolve()
# reduced_model.write("reduced_model.lp")

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Presolve removed 5 rows and 3 columns

Continuous model is non-convex -- presolving as a MIP

Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
CPU times: user 1.38 ms, sys: 1.47 ms, total: 2.85 ms
Wall time: 2.4 ms


In [43]:
full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
full_model.setParam('OutputFlag', 1)
# full_model.setParam('Presolve', -1)
full_model.optimize()
optimal_val_full = full_model.objVal
time_full = full_model.Runtime
optimal_x_full = np.array([v.x for v in full_model.getVars()])
# duals_full = np.array([c.Pi for c in full_model.getConstrs()])
# duals_ineq_target = duals_full[-G.shape[0]:]
# duals_eq_target = duals_full[:A.shape[0]]
active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
active_ineq_target.tolist()

active_lower_bounds = np.where(optimal_x_full - Lb_np <= 1e-4)[0]
active_upper_bounds = np.where(Ub_np - optimal_x_full <= 1e-4)[0]
print(active_ineq_target)
print(active_lower_bounds)
print(active_upper_bounds)

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 14 rows, 20 columns and 49 nonzeros
Model fingerprint: 0xaa73519e
Model has 64 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-02, 4e+00]
  Objective range  [3e-01, 4e+00]
  QObjective range [7e-04, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 6e+00]
Presolve removed 5 rows and 3 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolved: 112 rows, 79 columns, 329 nonzeros
Presolved model has 3 quadratic constraint(s)
Presolved model has 58 bilinear constraint(s)
Variable types: 79 continuous, 0 integer (0 binary)
Found heuristi

In [84]:
inactive_lower_bounds = np.where(optimal_x_full - Lb_np > 1e-4)[0]
inactive_upper_bounds = np.where(Ub_np - optimal_x_full > 1e-4)[0]

In [14]:
# calculate the parameter values
optimal_val_full = 13
# num_ineq = 11424 + 7024*2
opt_gap = 1e-2
# fac = max(1, max(np.abs(Q_np).max(), np.abs(p_np).max()))
fac = 1
t = min(fac * (num_ineq + num_var * 2) / opt_gap / abs(optimal_val_full), 1000)
dual_threshold = 1e-3
ineq_threshold = 1 / dual_threshold / (int(t))

print(int(t), ineq_threshold)

384 2.6041666666666665


In [15]:
(t / 5) ** (1/20)

1.2425184103674674

# remove part of the inequality constraints

In [87]:
# start_time = time.time()
# Yhat = model(X[idx].unsqueeze(0).to('cuda'))
# Ystar = data.projection(X[idx].unsqueeze(0).to('cuda'), Yhat)
# end_time = time.time()


In [88]:
start_time = time.time()
Yhat = model(X[idx].unsqueeze(0))
Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
end_time = time.time()
inference_time = end_time - start_time
print("Inference time: ", inference_time)
Ystar = Ystar.detach().numpy().flatten()
ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
ineq_sorted = np.sort(-G_np@Ystar + h_np)
print(ineq_sorted)
print(ineq_sorted_index)
inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]
print(inactive_ineq_index)

lb_sorted_index = np.argsort(Ystar - Lb_np)
lb_sorted = np.sort(Ystar - Lb_np)
inactive_lower_bounds = lb_sorted_index[lb_sorted > ineq_threshold]

ub_sorted_index = np.argsort(Ub_np - Ystar)
ub_sorted = np.sort(Ub_np - Ystar)
inactive_upper_bounds = ub_sorted_index[ub_sorted > ineq_threshold]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 1024])

In [62]:
set(inactive_ineq_index).intersection(set(active_ineq_target))

set()

In [63]:
remaining_ineq_index = np.setdiff1d(np.arange(num_ineq), inactive_ineq_index)
G_remaining = G[remaining_ineq_index]
h_remaining = h[remaining_ineq_index]

lb_remaining = Lb_np.copy()
ub_remaining = Ub_np.copy()
lb_remaining[inactive_lower_bounds] = -np.inf
ub_remaining[inactive_upper_bounds] = np.inf

In [64]:
model_reduced = build_gurobi_model(Q, p, A, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
model_reduced.setParam('OutputFlag', 1)
model_reduced.setParam('Presolve', -1)
# initialize the primal variables in model_hat2
# for i, v in enumerate(model_hat2.getVars()):
#     v.setAttr('Start', Ystar[i])
model_reduced.optimize()
optimal_val = model_reduced.objVal
time_reduced = model_reduced.Runtime + inference_time
optimal_x = np.array([v.x for v in model_reduced.getVars()])
print("Value: full={}, reduced={}, diff={}".format(optimal_val_full, optimal_val, optimal_val_full - optimal_val))
print("Time: full={}, reduced={}, diff={}".format(time_full, time_reduced, time_full - time_reduced))

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 936 rows, 483 columns and 2244 nonzeros
Model fingerprint: 0x0581cbef
Model has 31 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-01, 3e+02]
  Objective range  [7e+02, 2e+03]
  QObjective range [4e+01, 4e+01]
  Bounds range     [7e-03, 2e+00]
  RHS range        [3e-03, 8e-01]
Presolve removed 758 rows and 392 columns
Presolve time: 0.00s
Presolved: 178 rows, 209 columns, 755 nonzeros
Presolved model has 31 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.953e+03
 Factor NZ  : 4.026e+03
 Factor Ops : 1.026e+05 (less than 1 second per iteration)
 Threads    : 1

        

In [65]:
full_model.Runtime, model_reduced.Runtime

(0.010710000991821289, 0.009763002395629883)

# osqp warm start

In [89]:
num_trial = 10

total_time = 0.0
for _ in range(num_trial):
    solver = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    # solver1.warm_start(x=Ystar)
    start_time = time.time()
    results = solver.solve()
    end_time = time.time()
    total_time += (end_time - start_time)
print(total_time / num_trial)
print(results.info.obj_val)


ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


ValueError: Workspace allocation error!

In [30]:
total_time_ws = 0.0
for _ in range(num_trial):
    solver_ws = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    solver_ws.warm_start(x=Ystar)
    start_time = time.time()
    result_ws = solver_ws.solve()
    end_time = time.time()
    total_time_ws += (end_time - start_time)
print(total_time_ws / num_trial + inference_time)
print(result_ws.info.obj_val)

0.016102910041809082
13375.848118813015


In [31]:
total_time_reduced = 0.0
for _ in range(num_trial):
    solver_reduced = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
    start_time = time.time()
    results_reduced = solver_reduced.solve()
    end_time = time.time()
    total_time_reduced = end_time - start_time
print(total_time_reduced / num_trial + inference_time)
print(results_reduced.info.obj_val)

0.0038568258285522463
13375.833319200805


In [32]:
total_time_reduced_ws = 0.0
for _ in range(num_trial):
    solver_reduced_ws = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
    solver_reduced_ws.warm_start(x=Ystar)
    start_time = time.time()
    results_reduced_ws = solver_reduced_ws.solve()
    end_time = time.time()
    total_time_reduced_ws = end_time - start_time
print(total_time_reduced_ws / num_trial + inference_time)
print(results_reduced_ws.info.obj_val)

0.003991389274597168
13375.773211036943


In [33]:
x = results_reduced.x
data.ineq_dist(X_np[idx], torch.tensor(x).unsqueeze(0)).max()

tensor(5.0846e-09)

# check

In [23]:
error = []
for i in range(9900, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    full_model.optimize()
    optimal_x_full = np.array([v.x for v in full_model.getVars()])
    active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
    active_ineq_target.tolist()

    Yhat = model(X[idx].unsqueeze(0))
    Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
    Ystar = Ystar.detach().numpy().flatten()
    ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
    ineq_sorted = np.sort(-G_np@Ystar + h_np)
    inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]

    error.append(len(set(inactive_ineq_index).intersection(set(active_ineq_target))))
print(max(error))



0


In [91]:
success = 0
for i in range(9900, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    full_model.optimize()
    if full_model.status == GRB.OPTIMAL:
        success += 1
print(success)

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 14 rows, 20 columns and 49 nonzeros
Model fingerprint: 0xbf171bbe
Model has 64 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-02, 4e+00]
  Objective range  [3e-01, 4e+00]
  QObjective range [1e-03, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 6e+00]
Presolve removed 5 rows and 3 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 5 rows and 3 columns
Presolve time: 0.00s
Presolved: 112 rows, 79 columns, 329 nonzeros
Presolved model has 3 quadratic constraint(s)
Presolved model has 58 bilinear constraint(s)
Variable types: 79 continuous, 0 integer (0 binary)
Found heuristi